In [13]:
!pip install gurobipy


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gurobipy as gp
from gurobipy import Model, GRB, quicksum
import time
import random

In [15]:
semilla = [1, 2, 3, 4, 5, 6]
lista = []

# Dimensiones
F = 5  # Plantas
P = 5  # Productos
M = 100  # Mercados
S = 12  # Centros de distribución
K_OPEN = 6  # máximo CDs abiertos
DEBUG = False  # pon True si quieres ver prints de diagnóstico por semilla

for seed in semilla:
    np.random.seed(seed)

    # ------------------------
    # Generación de datos
    # ------------------------
    coords_f = np.random.uniform(0, 1001, size=(F, 2))
    coords_s = np.random.uniform(0, 1001, size=(S, 2))
    coords_m = np.random.uniform(0, 1001, size=(M, 2))

    demand = np.random.uniform(2000, 12000, size=(M, P))  # M x P
    total_demand = demand.sum(axis=0)                      # vector P
    total_demand_all = float(total_demand.sum())           # escalar

    # Capacidad de planta por producto (proporcional a la demanda de ese producto)
    cap_fp = np.zeros((F, P), dtype=float)
    for p in range(P):
        proporciones = np.random.dirichlet(np.ones(F))
        cap_fp[:, p] = proporciones * total_demand[p] * 1.2  # 20% holgura base

    # Capacidad total de CDs (proporcional a la demanda total)
    cap_s = np.random.dirichlet(np.ones(S)) * total_demand_all * 1.2  # vector S (float)

    cap_fp *= 2.0
    cap_s  *= 2.0
    # ------------------------
    # Ajustes de factibilidad (clave)
    # ------------------------

    # (A) Asegurar capacidad de PLANTAS por producto (capacidad >= demanda de ese producto)
    #for p in range(P):
        #cap_p = cap_fp[:, p].sum()
        #if cap_p < total_demand[p]:
            #factor = 1.05 * (total_demand[p] / max(1e-12, cap_p))  # +5% de holgura
            #cap_fp[:, p] *= factor

    # (B) Asegurar capacidad TOTAL de CDs (suma cap_s >= demanda total)
    #cap_tot_cds = cap_s.sum()
    #if cap_tot_cds < total_demand_all:
        #factor = 1.05 * (total_demand_all / max(1e-12, cap_tot_cds))
        #cap_s *= factor

    # (C) Asegurar que con el límite de 6 CDs exista un subconjunto factible:
    #     suma de las 6 mayores capacidades >= demanda total
    #cap_topk = np.sort(cap_s)[-K_OPEN:].sum()
    #if cap_topk < total_demand_all:
        #factor = 1.05 * (total_demand_all / max(1e-12, cap_topk))
        #cap_s *= factor  # escala todo; garantiza que top-6 puedan cubrir la demanda

    #if DEBUG:
        #print(f"[Seed {seed}] Demanda total: {total_demand_all:,.0f} | "
             # f"Cap.Plantas tot: {cap_fp.sum():,.0f} | "
             # f"Cap.CDs tot: {cap_s.sum():,.0f} | "
              #f"Top-{K_OPEN} CDs: {np.sort(cap_s)[-K_OPEN:].sum():,.0f}")
        #for p in range(P):
            #print(f"  - Prod {p+1}: Dem {total_demand[p]:,.0f} vs Cap {cap_fp[:,p].sum():,.0f}")

    # ------------------------
    # Costos
    # ------------------------
    f_s = np.random.uniform(300000, 900000, size=S)
    factor_p = np.random.uniform(0.8, 1.2, size=P)

    c1_fsp = np.zeros((F, S, P), dtype=float)
    c2_smp = np.zeros((S, M, P), dtype=float)
    for p in range(P):
        for f in range(F):
            for s in range(S):
                dist_fs = np.linalg.norm(coords_f[f] - coords_s[s])
                c1_fsp[f, s, p] = factor_p[p] * dist_fs
        for s in range(S):
            for m in range(M):
                dist_sm = np.linalg.norm(coords_s[s] - coords_m[m])
                c2_smp[s, m, p] = factor_p[p] * dist_sm

    # ------------------------
    # Modelo MILP (Gurobi)
    # ------------------------
    model = Model("CapacitatedFacilityLocation")
    model.setParam("OutputFlag", 0)

    Y = model.addVars(S, vtype=GRB.BINARY, name="Y")                # abrir CD
    X = model.addVars(F, S, P, vtype=GRB.CONTINUOUS, name="X")      # planta→CD
    Kvar = model.addVars(S, M, P, vtype=GRB.CONTINUOUS, name="K")   # CD→mercado
    W = model.addVars(S, M, P, vtype=GRB.BINARY, name="W")

    model.setObjective(
        quicksum(f_s[s] * Y[s] for s in range(S)) +
        quicksum(c1_fsp[f, s, p] * X[f, s, p] for f in range(F) for s in range(S) for p in range(P)) +
        quicksum(c2_smp[s, m, p] * Kvar[s, m, p] for s in range(S) for m in range(M) for p in range(P)),
        GRB.MINIMIZE
    )

    # Demanda por mercado y producto
    for m in range(M):
        for p in range(P):
            model.addConstr(quicksum(Kvar[s, m, p] for s in range(S)) == demand[m, p])

    # Single-sourcing: cada (m,p) desde un único s
    for m in range(M):
        for p in range(P):
             model.addConstr(gp.quicksum(W[s, m, p] for s in range(S)) == 1,
                        name=f"unique[{m},{p}]")

    # Vínculo: Kvar[s,m,p] ≤ d[m,p] * W[s,m,p]
    for s in range(S):
        for m in range(M):
            for p in range(P):
               model.addConstr(Kvar[s, m, p] <= float(demand[m, p]) * W[s, m, p],
                            name=f"link[{s},{m},{p}]")

    # Si el CD no está abierto, no se puede asignar (m,p) a s
    for s in range(S):
        for m in range(M):
            for p in range(P):
                model.addConstr(W[s, m, p] <= Y[s], name=f"open_for_assign[{s},{m},{p}]")

    # Capacidad de plantas por producto
    for f in range(F):
        for p in range(P):
            model.addConstr(quicksum(X[f, s, p] for s in range(S)) <= cap_fp[f, p])

    # Balance en CD por producto (permitimos sobrestock: ≥)
    for s in range(S):
        for p in range(P):
            model.addConstr(quicksum(X[f, s, p] for f in range(F)) >= quicksum(Kvar[s, m, p] for m in range(M)))

    # Capacidad de CD (agregada en todos los productos)
    for s in range(S):
        model.addConstr(quicksum(Kvar[s, m, p] for m in range(M) for p in range(P)) <= cap_s[s] * Y[s])

    # Límite de CDs abiertos
    model.addConstr(quicksum(Y[s] for s in range(S)) <= K_OPEN)

    # ------------------------
    # Resolver
    # ------------------------
    start_time = time.process_time()
    model.optimize()
    elapsed_time = time.process_time() - start_time

    # ------------------------
    # Guardar resultados
    # ------------------------
    if model.status == GRB.OPTIMAL:
        cds_abiertos = sum(1 for s in range(S) if Y[s].X > 0.5)
        costo_apertura    = sum(f_s[s] * Y[s].X for s in range(S))
        costo_planta_cd   = sum(c1_fsp[f, s, p] * X[f, s, p].X for f in range(F) for s in range(S) for p in range(P))
        costo_cd_mercado  = sum(c2_smp[s, m, p] * Kvar[s, m, p].X for s in range(S) for m in range(M) for p in range(P))
        lista.append({
            "semilla": seed,
            "cds_abiertos": cds_abiertos,
            "costo_apertura": costo_apertura,
            "costo_planta_cd": costo_planta_cd,
            "costo_cd_mercado": costo_cd_mercado,
            "costo_total": model.ObjVal,
            "tiempo_solucion": elapsed_time,
            "status": "OPTIMAL"
        })
    else:
        # Si por alguna razón sigue infactible, deja rastro útil
        if DEBUG:
            print(f"[Seed {seed}] INFEASIBLE. top-{K_OPEN} cap: {np.sort(cap_s)[-K_OPEN:].sum():,.0f}, "
                  f"totCD: {cap_s.sum():,.0f}, totDem: {total_demand_all:,.0f}")
        lista.append({
            "semilla": seed,
            "cds_abiertos": None,
            "costo_apertura": None,
            "costo_planta_cd": None,
            "costo_cd_mercado": None,
            "costo_total": None,
            "tiempo_solucion": elapsed_time,
            "status": "INFEASIBLE"
        })

In [16]:
# ✅ Validaciones para los datos del modelo (consistencia estructural y lógica)

# 1. Validar que para cada producto, la suma de capacidad de plantas ≥ demanda
for p in range(P):
    total_cap = cap_fp[:, p].sum()
    total_dem = demand[:, p].sum()
    assert total_cap > total_dem, f"❌ Capacidad insuficiente para producto {p+1} (cap={total_cap}, dem={total_dem})"

# 2. Validar que la capacidad total de los CDs cubra la demanda global
assert cap_s.sum() > demand.sum(), "❌ Capacidad total de CDs insuficiente"

# 3. Validar dimensiones de matrices
assert coords_f.shape == (F, 2), "❌ coords_f tiene dimensiones incorrectas"
assert coords_s.shape == (S, 2), "❌ coords_s tiene dimensiones incorrectas"
assert coords_m.shape == (M, 2), "❌ coords_m tiene dimensiones incorrectas"
assert demand.shape == (M, P), "❌ demand tiene dimensiones incorrectas"
assert cap_fp.shape == (F, P), "❌ cap_fp tiene dimensiones incorrectas"
assert cap_s.shape == (S,), "❌ cap_s tiene dimensiones incorrectas"
assert f_s.shape == (S,), "❌ f_s tiene dimensiones incorrectas"
assert c1_fsp.shape == (F, S, P), "❌ c1_fsp tiene dimensiones incorrectas"
assert c2_smp.shape == (S, M, P), "❌ c2_smp tiene dimensiones incorrectas"

# 4. Validar que todos los costos sean ≥ 0
assert np.all(c1_fsp > 0), "❌ Hay costos negativos en c1_fsp"
assert np.all(c2_smp > 0), "❌ Hay costos negativos en c2_smp"
assert np.all(f_s > 0), "❌ Hay costos fijos negativos"

# 5. Validar que no hay NaN o inf en ningún dato
for name, array in {
    "coords_f": coords_f,
    "coords_s": coords_s,
    "coords_m": coords_m,
    "demand": demand,
    "cap_fp": cap_fp,
    "cap_s": cap_s,
    "f_s": f_s,
    "c1_fsp": c1_fsp,
    "c2_smp": c2_smp
}.items():
    assert np.isfinite(array).all(), f"❌ {name} contiene NaN o inf"

print("✅ Todas las validaciones fueron exitosas. Datos listos para el modelo 💪")

✅ Todas las validaciones fueron exitosas. Datos listos para el modelo 💪


In [17]:
print(lista)

[{'semilla': 1, 'cds_abiertos': 6, 'costo_apertura': np.float64(3772450.354688172), 'costo_planta_cd': np.float64(978483439.8227501), 'costo_cd_mercado': np.float64(922487161.8551507), 'costo_total': 1904743052.032589, 'tiempo_solucion': 58.875, 'status': 'OPTIMAL'}, {'semilla': 2, 'cds_abiertos': 6, 'costo_apertura': np.float64(4063640.15123934), 'costo_planta_cd': np.float64(477462864.99392825), 'costo_cd_mercado': np.float64(762928706.2378404), 'costo_total': 1244455211.3830075, 'tiempo_solucion': 6.953125, 'status': 'OPTIMAL'}, {'semilla': 3, 'cds_abiertos': 6, 'costo_apertura': np.float64(4151864.3240913544), 'costo_planta_cd': np.float64(592964176.7491703), 'costo_cd_mercado': np.float64(775244122.1181076), 'costo_total': 1372360163.1913688, 'tiempo_solucion': 7.421875, 'status': 'OPTIMAL'}, {'semilla': 4, 'cds_abiertos': 6, 'costo_apertura': np.float64(3150935.8524763267), 'costo_planta_cd': np.float64(723898380.7741845), 'costo_cd_mercado': np.float64(860410500.0683832), 'costo

In [18]:
for resultado in lista:
    print(f"📦 Resultados para semilla {resultado.get('semilla', '—')}")
    print("────────────────────────────────────────────")

    status = resultado.get("status", "UNKNOWN")

    print(f"🔎 Estado del modelo      : {status}")

    if status.upper() == "OPTIMAL":
        print(f"🏭 CDs abiertos           : {resultado['cds_abiertos']}")
        print(f"💰 Costo apertura CDs     : ${resultado['costo_apertura']:,.0f}")
        print(f"🚚 Costo planta → CD      : ${resultado['costo_planta_cd']:,.0f}")
        print(f"📦 Costo CD → mercado     : ${resultado['costo_cd_mercado']:,.0f}")
        print(f"🧾 Costo total            : ${resultado['costo_total']:,.0f}")
        print(f"⏱️ Tiempo de solución     : {resultado['tiempo_solucion']:.2f} s")
    else:
        print("⚠️ Modelo no resuelto óptimamente (inviable, ilimitado o error)")
        print(f"⏱️ Tiempo de intento      : {resultado.get('tiempo_solucion', '—'):.2f} s")

    print("────────────────────────────────────────────\n")

📦 Resultados para semilla 1
────────────────────────────────────────────
🔎 Estado del modelo      : OPTIMAL
🏭 CDs abiertos           : 6
💰 Costo apertura CDs     : $3,772,450
🚚 Costo planta → CD      : $978,483,440
📦 Costo CD → mercado     : $922,487,162
🧾 Costo total            : $1,904,743,052
⏱️ Tiempo de solución     : 58.88 s
────────────────────────────────────────────

📦 Resultados para semilla 2
────────────────────────────────────────────
🔎 Estado del modelo      : OPTIMAL
🏭 CDs abiertos           : 6
💰 Costo apertura CDs     : $4,063,640
🚚 Costo planta → CD      : $477,462,865
📦 Costo CD → mercado     : $762,928,706
🧾 Costo total            : $1,244,455,211
⏱️ Tiempo de solución     : 6.95 s
────────────────────────────────────────────

📦 Resultados para semilla 3
────────────────────────────────────────────
🔎 Estado del modelo      : OPTIMAL
🏭 CDs abiertos           : 6
💰 Costo apertura CDs     : $4,151,864
🚚 Costo planta → CD      : $592,964,177
📦 Costo CD → mercado     : $